# Handwriting Synthesis Model Training on Google Colab

This notebook trains a handwriting synthesis model using the `finnbusse/v3testing` dataset from Hugging Face.

## Features:
- Loads stroke data from HuggingFace Parquet format
- Optimized hyperparameters for different dataset sizes (565, 750, 1000, 1250, 1500, 1750, 2000 words)
- tqdm progress bars for training visualization
- Sample generation every 10 epochs
- Training graphs and model saving at the end

**Note:** Make sure to select a GPU runtime: Runtime > Change runtime type > GPU

## 1. Setup and Installation

In [ ]:
# Clone the repository
!git clone https://github.com/finnbusse/pytorch-handwriting-synthesis-toolkit.git
%cd pytorch-handwriting-synthesis-toolkit

In [ ]:
# Install dependencies (compatible with Google Colab Python 3.11+ and PyTorch 2.x)
# Note: Colab already has torch, numpy, matplotlib pre-installed
!pip install -q Pillow h5py svgwrite datasets tqdm

In [ ]:
import os
import sys
import json
import math
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from IPython.display import display, Image, clear_output
from datasets import load_dataset
import h5py

# Add the repository to the path
sys.path.insert(0, os.getcwd())

from handwriting_synthesis import data, models, utils, losses, metrics
from handwriting_synthesis.data import Tokenizer
from handwriting_synthesis.sampling import HandwritingSynthesizer
from handwriting_synthesis.tasks import HandwritingSynthesisTask
from handwriting_synthesis.optimizers import CustomRMSprop

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Configuration

Select your dataset size and corresponding optimized hyperparameters.

In [ ]:
# =============================================================================
# CONFIGURATION - Optimized hyperparameters for different dataset sizes
# =============================================================================

# Dataset settings
HUGGINGFACE_DATASET = "finnbusse/v3testing"

# Select your dataset size (approximate number of words)
# Options: 565, 750, 1000, 1250, 1500, 1750, 2000
DATASET_SIZE = 565  # Change this to match your dataset size

# Optimized hyperparameters for each dataset size
# These are carefully tuned for handwriting synthesis on small-to-medium datasets
HYPERPARAMETERS = {
    565: {
        'batch_size': 16,          # Larger batch for faster training on GPU
        'epochs': 200,             # More epochs for small dataset
        'learning_rate': 0.0001,   # Standard LR
        'hidden_size': 400,        # Standard hidden size
        'gradient_clip_output': 100,
        'gradient_clip_lstm': 10,
        'validation_split': 0.15,  # 15% for validation
        'max_seq_length': 500,     # Max points per stroke sequence
        'sample_interval': 5,      # Show sample every N epochs
    },
    750: {
        'batch_size': 20,
        'epochs': 180,
        'learning_rate': 0.0001,
        'hidden_size': 400,
        'gradient_clip_output': 100,
        'gradient_clip_lstm': 10,
        'validation_split': 0.15,
        'max_seq_length': 500,
        'sample_interval': 5,
    },
    1000: {
        'batch_size': 24,
        'epochs': 150,
        'learning_rate': 0.0001,
        'hidden_size': 400,
        'gradient_clip_output': 100,
        'gradient_clip_lstm': 10,
        'validation_split': 0.12,
        'max_seq_length': 500,
        'sample_interval': 5,
    },
    1250: {
        'batch_size': 28,
        'epochs': 130,
        'learning_rate': 0.0001,
        'hidden_size': 400,
        'gradient_clip_output': 100,
        'gradient_clip_lstm': 10,
        'validation_split': 0.12,
        'max_seq_length': 500,
        'sample_interval': 5,
    },
    1500: {
        'batch_size': 32,
        'epochs': 120,
        'learning_rate': 0.0001,
        'hidden_size': 400,
        'gradient_clip_output': 100,
        'gradient_clip_lstm': 10,
        'validation_split': 0.10,
        'max_seq_length': 500,
        'sample_interval': 5,
    },
    1750: {
        'batch_size': 32,
        'epochs': 110,
        'learning_rate': 0.0001,
        'hidden_size': 400,
        'gradient_clip_output': 100,
        'gradient_clip_lstm': 10,
        'validation_split': 0.10,
        'max_seq_length': 500,
        'sample_interval': 5,
    },
    2000: {
        'batch_size': 32,
        'epochs': 100,
        'learning_rate': 0.0001,
        'hidden_size': 400,
        'gradient_clip_output': 100,
        'gradient_clip_lstm': 10,
        'validation_split': 0.10,
        'max_seq_length': 500,
        'sample_interval': 5,
    },
}

# Get hyperparameters for selected dataset size (use closest match)
available_sizes = sorted(HYPERPARAMETERS.keys())
selected_size = min(available_sizes, key=lambda x: abs(x - DATASET_SIZE))
config = HYPERPARAMETERS[selected_size]

print(f"Selected configuration for ~{selected_size} words:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 3. Load Dataset from HuggingFace

In [ ]:
# Load the dataset from HuggingFace
print(f"Loading dataset: {HUGGINGFACE_DATASET}")
try:
    dataset = load_dataset(HUGGINGFACE_DATASET)
except Exception as e:
    print(f"Error loading dataset: {e}")
    print("\nPossible causes:")
    print("  - Dataset not found or requires authentication")
    print("  - Network connectivity issues")
    print("  - Invalid dataset name")
    print("\nPlease verify the dataset name and try again.")
    raise

# Display dataset info
print(f"\nDataset structure:")
print(dataset)

# Get the training split (or the main split if no train split exists)
if 'train' in dataset:
    raw_data = dataset['train']
else:
    # Use the first available split
    split_name = list(dataset.keys())[0]
    raw_data = dataset[split_name]

print(f"\nTotal samples: {len(raw_data)}")
print(f"\nSample data structure (first item):")
sample = raw_data[0]
for key in sample.keys():
    value = sample[key]
    if isinstance(value, (list, dict)):
        print(f"  {key}: {type(value).__name__} (length: {len(value) if hasattr(value, '__len__') else 'N/A'})")
    else:
        print(f"  {key}: {value}")

## 4. Data Conversion and Preprocessing

The dataset already contains preprocessed delta/offset data (`dx`, `dy`, `eos`), which is the format needed for training.

In [ ]:
def prepare_training_data(raw_data, max_seq_length=500):
    """
    Prepare training data from HuggingFace dataset.
    
    The finnbusse/v3testing dataset format:
      - dx: list of x deltas (offsets)
      - dy: list of y deltas (offsets)
      - eos: list of end-of-stroke flags (0 or 1)
      - scale: scale factor used during preprocessing
      - text: the word that was written
    
    This is already in the offset format expected by the training pipeline!
    We just need to combine dx, dy, eos into (dx, dy, eos) tuples.
    
    Returns:
        List of (offsets, text) tuples where offsets is a list of (dx, dy, eos) tuples
    """
    prepared_data = []
    skipped_empty = 0
    skipped_long = 0
    
    for item in tqdm(raw_data, desc="Converting data"):
        text = item.get('text', '')
        dx = item.get('dx', [])
        dy = item.get('dy', [])
        eos = item.get('eos', [])
        
        # Skip if no text or no stroke data
        if not text or not dx or not dy or not eos:
            skipped_empty += 1
            continue
        
        # Ensure all lists have same length
        min_len = min(len(dx), len(dy), len(eos))
        if min_len == 0:
            skipped_empty += 1
            continue
        
        # Check sequence length
        if min_len > max_seq_length:
            skipped_long += 1
            continue
        
        # Combine into offset tuples (dx, dy, eos)
        offsets = [(float(dx[i]), float(dy[i]), int(eos[i])) for i in range(min_len)]
        
        # Make sure last point has eos=1
        if offsets:
            last = offsets[-1]
            offsets[-1] = (last[0], last[1], 1)
        
        prepared_data.append((offsets, text))
    
    if skipped_empty > 0:
        print(f"Skipped {skipped_empty} samples with empty data")
    if skipped_long > 0:
        print(f"Skipped {skipped_long} samples exceeding max length {max_seq_length}")
    
    return prepared_data


# Prepare the data
print("Preparing training data...")
all_data = prepare_training_data(raw_data, config['max_seq_length'])
print(f"\nPrepared {len(all_data)} valid samples")

# Show a sample
if all_data:
    sample_offsets, sample_text = all_data[0]
    print(f"\nSample:")
    print(f"  Text: '{sample_text}'")
    print(f"  Sequence length: {len(sample_offsets)}")
    print(f"  First 5 offsets: {sample_offsets[:5]}")
else:
    print("\nWARNING: No valid samples found! Check dataset format.")
    print(f"Dataset columns: {raw_data.column_names}")
    if len(raw_data) > 0:
        print(f"First item keys: {list(raw_data[0].keys())}")

In [ ]:
# Split data into training and validation sets
import random
random.seed(42)

# Shuffle the data
shuffled_data = all_data.copy()
random.shuffle(shuffled_data)

# Split
val_size = int(len(shuffled_data) * config['validation_split'])
train_size = len(shuffled_data) - val_size

train_data = shuffled_data[:train_size]
val_data = shuffled_data[train_size:]

print(f"Training samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")

## 5. Build Character Set (Charset)

In [ ]:
# Build charset from training data
def build_charset(data):
    """Build character set from the text data."""
    charset = set()
    for _, text in data:
        charset.update(set(text))
    # Sort for consistency
    return ''.join(sorted(charset))

charset = build_charset(train_data)
print(f"Charset ({len(charset)} characters): '{charset}'")

# Create tokenizer
tokenizer = Tokenizer(charset)
print(f"Tokenizer size (with padding): {tokenizer.size}")

## 6. Create H5 Dataset Files

In [ ]:
# Create data directory
DATA_DIR = 'colab_data'
os.makedirs(DATA_DIR, exist_ok=True)

def save_offsets_to_h5(data_list, save_path, max_length):
    """
    Save offset data to H5 format.
    
    The data is already in offset format (dx, dy, eos), so we just need to:
    1. Truncate if necessary
    2. Save to H5 with computed mean and std
    """
    processed = []
    
    for offsets, text in tqdm(data_list, desc=f"Processing {os.path.basename(save_path)}"):
        # Truncate if needed
        truncated = data.truncate_sequence(offsets, max_length)
        
        if truncated:
            processed.append((truncated, text))
    
    # Save to H5
    data.save_to_h5(processed, save_path, max_length)
    
    return len(processed)


# Determine max sequence length from data
max_lengths = [len(offsets) for offsets, _ in train_data]

actual_max_length = max(max_lengths) if max_lengths else config['max_seq_length']
max_length = min(actual_max_length, config['max_seq_length'])
print(f"Maximum sequence length in data: {actual_max_length}")
print(f"Using max length: {max_length}")

# Process and save training data
train_path = os.path.join(DATA_DIR, 'train.h5')
num_train = save_offsets_to_h5(train_data, train_path, max_length)
print(f"Saved {num_train} training examples to {train_path}")

# Process and save validation data
val_path = os.path.join(DATA_DIR, 'val.h5')
num_val = save_offsets_to_h5(val_data, val_path, max_length)
print(f"Saved {num_val} validation examples to {val_path}")

# Save charset
charset_path = os.path.join(DATA_DIR, 'charset.txt')
tokenizer.save_charset(charset_path)
print(f"Saved charset to {charset_path}")

## 7. Load Prepared Datasets

In [ ]:
# Load the prepared datasets
with data.H5Dataset(train_path) as train_h5:
    mu = train_h5.mu
    std = train_h5.std
    dataset_max_length = train_h5.max_length

print(f"Dataset statistics:")
print(f"  Mean (mu): {mu}")
print(f"  Std: {std}")
print(f"  Max length: {dataset_max_length}")

## 8. Training Setup

Set up the model, optimizer, and training components.

In [ ]:
# Get device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create model
alphabet_size = tokenizer.size
model = models.SynthesisNetwork.get_default_model(alphabet_size, device)
model = model.to(device)

print(f"\nModel created:")
print(f"  Alphabet size: {alphabet_size}")
print(f"  Hidden size: {model.hidden_size}")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")

In [ ]:
# Create synthesizer for saving/loading
mu_tensor = torch.tensor(mu, dtype=torch.float32)
sd_tensor = torch.tensor(std, dtype=torch.float32)

synthesizer = HandwritingSynthesizer(
    model, mu_tensor, sd_tensor, charset, num_steps=dataset_max_length
)

# Create optimizer
# Note: momentum=9000 is used as a flag to enable momentum buffer in CustomRMSprop.
# The actual momentum decay is hardcoded to 0.9 in the optimizer implementation.
# This follows the original training code from the repository.
optimizer = CustomRMSprop(
    model.parameters(),
    lr=config['learning_rate'],
    alpha=0.95,
    eps=1e-4,
    momentum=9000,
    centered=True
)

# Gradient clipping values
clip_output = config['gradient_clip_output']
clip_lstm = config['gradient_clip_lstm']

print("Optimizer and synthesizer created")

## 9. Training Loop with tqdm and Sample Visualization

In [ ]:
def collate_fn(batch):
    """Collate function for DataLoader."""
    x, y = [], []
    for points, text in batch:
        x.append(points)
        y.append(text)
    return x, y


def compute_loss(model, batch, tokenizer, device):
    """Compute loss for a batch."""
    points, transcriptions = batch
    ground_true = utils.PaddedSequencesBatch(points, device=device)
    
    batch_size, steps, input_dim = ground_true.tensor.shape
    
    # Create input (shifted by one step)
    prefix = torch.zeros(batch_size, 1, input_dim, device=device)
    x = torch.cat([prefix, ground_true.tensor[:, :-1]], dim=1)
    
    # Create context (one-hot encoded transcriptions)
    c = data.transcriptions_to_tensor(tokenizer, transcriptions).to(device)
    
    # Forward pass
    mixtures, eos_hat = model(x, c)
    
    # Compute loss
    loss = losses.nll_loss(mixtures, eos_hat, ground_true)
    
    return (mixtures, eos_hat), loss


def generate_sample(model, tokenizer, mu, std, text, device, max_steps=500):
    """
    Generate a handwriting sample for given text.
    
    Args:
        model: The SynthesisNetwork model
        tokenizer: Tokenizer for encoding text
        mu: Mean values for denormalization (tuple)
        std: Standard deviation values for denormalization (tuple)
        text: Text to synthesize
        device: PyTorch device
        max_steps: Maximum number of points to generate
    
    Returns:
        Tensor on CPU containing denormalized handwriting coordinates
    """
    model.eval()
    
    with torch.no_grad():
        # Encode text
        sentinel = '\n'
        text_with_sentinel = text + sentinel
        c = data.transcriptions_to_tensor(tokenizer, [text_with_sentinel]).to(device)
        
        # Generate
        sample = model.sample_means(context=c, steps=max_steps, stochastic=True)
        
        # Denormalize and move to CPU
        sample = sample.cpu() * torch.tensor(std) + torch.tensor(mu)
        
    model.train()
    return sample


def visualize_sample(sample, text, save_path=None):
    """
    Visualize a handwriting sample and display it.
    
    Handles cases where visualization fails (e.g., during early training
    when the model produces wild coordinates).
    """
    # Use provided save_path or create a default one
    if save_path is None:
        save_path = os.path.join(SAMPLES_DIR, 'temp_sample.png')
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    try:
        # Check if sample is valid
        if sample is None or len(sample) == 0:
            print(f"Text: '{text}' - [Empty sample, model still learning]")
            return
        
        # Get coordinates to check if they're reasonable
        sample_np = sample.cpu().numpy() if hasattr(sample, 'cpu') else sample
        x_range = sample_np[:, 0].max() - sample_np[:, 0].min() if len(sample_np) > 0 else 0
        y_range = sample_np[:, 1].max() - sample_np[:, 1].min() if len(sample_np) > 0 else 0
        
        # If coordinates are too wild (happens early in training), show a message
        if abs(x_range) > 5000 or abs(y_range) > 5000:
            print(f"Text: '{text}' - [Coordinates too wild: x_range={x_range:.0f}, y_range={y_range:.0f}. Model still learning, this is normal early in training.]")
            return
        
        # Try to visualize
        utils.visualize_strokes(sample, save_path, lines=True, thickness=8)
        
        # Check if file was created and has content
        if os.path.exists(save_path) and os.path.getsize(save_path) > 0:
            display(Image(filename=save_path))
            print(f"Text: '{text}'")
        else:
            print(f"Text: '{text}' - [Image not generated - model predictions may be too noisy. This is normal early in training.]")
            
    except Exception as e:
        print(f"Text: '{text}' - [Visualization failed: {str(e)[:100]}. Model still learning.]")

In [ ]:
# Training configuration
EPOCHS = config['epochs']
BATCH_SIZE = config['batch_size']
SAMPLE_INTERVAL = config['sample_interval']  # Show samples every N epochs
MODEL_SAVE_DIR = 'colab_checkpoints'
SAMPLES_DIR = 'colab_samples'

os.makedirs(MODEL_SAVE_DIR, exist_ok=True)
os.makedirs(SAMPLES_DIR, exist_ok=True)

print(f"Training Configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Sample every: {SAMPLE_INTERVAL} epochs")
print(f"  Model save directory: {MODEL_SAVE_DIR}")
print(f"  Samples directory: {SAMPLES_DIR}")

In [ ]:
# Training loop
print("\n" + "="*60)
print("Starting Training")
print("="*60 + "\n")

# Training history
train_losses = []
val_losses = []

# Initialize best_model_dir before training loop
best_model_dir = os.path.join(MODEL_SAVE_DIR, 'best_model')

# Open datasets
train_dataset = data.NormalizedDataset(train_path, mu, std)
val_dataset = data.NormalizedDataset(val_path, mu, std)

# Validate datasets are not empty
if len(train_dataset) == 0:
    train_dataset.close()
    val_dataset.close()
    raise ValueError("Training dataset is empty! Please check your data preparation.")

if len(val_dataset) == 0:
    print("Warning: Validation dataset is empty. Training will proceed without validation.")

# Get sample texts for visualization
sample_texts = []
for i in range(min(3, len(val_dataset))):
    _, text = val_dataset[i]
    sample_texts.append(text)

print(f"Sample texts for visualization: {sample_texts}")

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

num_train_batches = len(train_loader)
num_val_batches = len(val_loader)

print(f"Training batches per epoch: {num_train_batches}")
print(f"Validation batches per epoch: {num_val_batches}")

# Training
best_val_loss = float('inf')

try:
    epoch_pbar = tqdm(range(1, EPOCHS + 1), desc="Training", unit="epoch")

    for epoch in epoch_pbar:
        # Training phase
        model.train()
        epoch_train_loss = 0.0
        
        batch_pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}", leave=False, unit="batch")
        
        for batch_idx, batch in enumerate(batch_pbar):
            optimizer.zero_grad()
            
            # Compute loss
            y_hat, loss = compute_loss(model, batch, tokenizer, device)
            
            # Backward pass
            loss.backward()
            
            # Gradient clipping
            model.clip_gradients(output_clip_value=clip_output, lstm_clip_value=clip_lstm)
            
            # Update weights
            optimizer.step()
            
            epoch_train_loss += loss.item()
            batch_pbar.set_postfix({'loss': f'{loss.item():.2f}'})
        
        avg_train_loss = epoch_train_loss / num_train_batches
        train_losses.append(avg_train_loss)
        
        # Validation phase
        model.eval()
        epoch_val_loss = 0.0
        
        with torch.no_grad():
            for batch in val_loader:
                _, loss = compute_loss(model, batch, tokenizer, device)
                epoch_val_loss += loss.item()
        
        avg_val_loss = epoch_val_loss / num_val_batches if num_val_batches > 0 else 0
        val_losses.append(avg_val_loss)
        
        # Update progress bar
        epoch_pbar.set_postfix({
            'train_loss': f'{avg_train_loss:.2f}',
            'val_loss': f'{avg_val_loss:.2f}'
        })
        
        # Save best model
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            synthesizer.save(best_model_dir)
        
        # Generate and display samples every SAMPLE_INTERVAL epochs
        if epoch % SAMPLE_INTERVAL == 0:
            print(f"\n\n{'='*60}")
            print(f"Epoch {epoch} - Sample Generation")
            print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")
            print(f"{'='*60}")
            
            for i, text in enumerate(sample_texts[:2]):  # Show 2 samples
                sample = generate_sample(model, tokenizer, mu, std, text, device, dataset_max_length)
                sample_path = os.path.join(SAMPLES_DIR, f'epoch_{epoch}_sample_{i}.png')
                visualize_sample(sample, text, sample_path)
            
            print()
        
        # Save checkpoint every 20 epochs
        if epoch % 20 == 0:
            checkpoint_dir = os.path.join(MODEL_SAVE_DIR, f'Epoch_{epoch}')
            synthesizer.save(checkpoint_dir)

    print("\n" + "="*60)
    print("Training Complete!")
    print(f"Best validation loss: {best_val_loss:.4f}")
    print("="*60)

finally:
    # Always close datasets to prevent resource leaks
    train_dataset.close()
    val_dataset.close()

## 10. Training Results Visualization

In [ ]:
# Plot training history
plt.figure(figsize=(12, 5))

# Loss plot
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss', color='blue', alpha=0.8)
plt.plot(val_losses, label='Validation Loss', color='orange', alpha=0.8)
plt.xlabel('Epoch')
plt.ylabel('Loss (nats)')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# Smoothed loss plot
plt.subplot(1, 2, 2)
window = min(10, len(train_losses) // 4) if len(train_losses) > 4 else 1
if window > 1:
    train_smooth = np.convolve(train_losses, np.ones(window)/window, mode='valid')
    val_smooth = np.convolve(val_losses, np.ones(window)/window, mode='valid')
    plt.plot(range(window-1, len(train_losses)), train_smooth, label='Training (smoothed)', color='blue')
    plt.plot(range(window-1, len(val_losses)), val_smooth, label='Validation (smoothed)', color='orange')
else:
    plt.plot(train_losses, label='Training Loss', color='blue')
    plt.plot(val_losses, label='Validation Loss', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Loss (nats)')
plt.title('Smoothed Training and Validation Loss')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_history.png', dpi=150)
plt.show()

print(f"\nFinal Training Loss: {train_losses[-1]:.4f}")
print(f"Final Validation Loss: {val_losses[-1]:.4f}")
print(f"Best Validation Loss: {best_val_loss:.4f}")

## 11. Generate Final Samples

In [ ]:
# Load best model
best_model_dir = os.path.join(MODEL_SAVE_DIR, 'best_model')
best_synthesizer = HandwritingSynthesizer.load(best_model_dir, device, bias=1.0)

print("Loaded best model for final samples\n")

# Generate samples for all validation texts
print("="*60)
print("Final Generated Samples")
print("="*60 + "\n")

# Reload validation data to get texts
val_dataset = data.NormalizedDataset(val_path, mu, std)

for i in range(min(5, len(val_dataset))):
    _, text = val_dataset[i]
    
    print(f"\nSample {i+1}: '{text}'")
    
    sample = generate_sample(
        best_synthesizer.model, 
        tokenizer, 
        mu, std, 
        text, 
        device, 
        dataset_max_length
    )
    
    sample_path = os.path.join(SAMPLES_DIR, f'final_sample_{i}.png')
    visualize_sample(sample, text, sample_path)

val_dataset.close()

print("\n" + "="*60)
print("All samples generated!")
print("="*60)

## 12. Save Final Model

In [ ]:
# Save final model
final_model_dir = os.path.join(MODEL_SAVE_DIR, 'final_model')
synthesizer.save(final_model_dir)

print(f"Final model saved to: {final_model_dir}")
print(f"Best model saved to: {best_model_dir}")

# List all saved files
print(f"\nSaved model files:")
for root, dirs, files in os.walk(MODEL_SAVE_DIR):
    level = root.replace(MODEL_SAVE_DIR, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        print(f"{subindent}{file}")

## 13. Download Model (Optional)

In [ ]:
# Zip the models for download
import shutil

# Create zip archive
shutil.make_archive('handwriting_model', 'zip', MODEL_SAVE_DIR)

print("Model archive created: handwriting_model.zip")
print("\nTo download, click on the file in the file browser (left sidebar) and download.")

# For Google Colab - download directly
try:
    from google.colab import files
    files.download('handwriting_model.zip')
except ImportError:
    print("Not running in Google Colab - manual download required.")

## 14. Interactive Synthesis (Optional)

Use this cell to generate handwriting for custom text.

In [ ]:
# Custom text synthesis
CUSTOM_TEXT = "Hallo"  # Change this to your desired text

print(f"Generating handwriting for: '{CUSTOM_TEXT}'")

# Check if all characters are in charset
missing_chars = set(CUSTOM_TEXT) - set(charset)
if missing_chars:
    print(f"Warning: Characters not in charset: {missing_chars}")
    print(f"Available characters: '{charset}'")
else:
    # Use best_synthesizer if available, otherwise fall back to model
    synthesis_model = best_synthesizer.model if 'best_synthesizer' in dir() and best_synthesizer is not None else model
    
    sample = generate_sample(
        synthesis_model,
        tokenizer,
        mu, std,
        CUSTOM_TEXT,
        device,
        dataset_max_length
    )
    
    custom_path = os.path.join(SAMPLES_DIR, 'custom_sample.png')
    visualize_sample(sample, CUSTOM_TEXT, custom_path)

---

## Training Summary

This notebook trained a handwriting synthesis model using:

- **Dataset**: HuggingFace dataset `finnbusse/v3testing`
- **Model**: SynthesisNetwork based on Alex Graves' paper "Generating Sequences With Recurrent Neural Networks"
- **Architecture**: 3-layer LSTM with soft attention window mechanism

### Hyperparameters Used
- Batch size: {config['batch_size']}
- Epochs: {config['epochs']}
- Learning rate: {config['learning_rate']}
- Hidden size: {config['hidden_size']}
- Gradient clipping: {config['gradient_clip_output']} (output), {config['gradient_clip_lstm']} (LSTM)

### Tips for Better Results
1. **More data**: Collect more handwriting samples for better generalization
2. **Longer training**: Increase epochs if validation loss is still decreasing
3. **Data augmentation**: Add slight variations to stroke data
4. **Probability bias**: Use bias parameter when generating samples for cleaner output